In [11]:
import solana
from solana.publickey import PublicKey
from solana.rpc.api import Client
from solana.keypair import Keypair
from solana import system_program
from solana import transaction
from solana.system_program import SYS_PROGRAM_ID, CreateAccountParams, create_account
from solana.transaction import *
from spl.token.constants import TOKEN_PROGRAM_ID, ASSOCIATED_TOKEN_PROGRAM_ID
from spl.token.instructions import initialize_mint, InitializeMintParams
from spl.token._layouts import MINT_LAYOUT
from spl.token.client import Token
from spl.token import instructions
import base64
from borsh_construct import *
from solana.rpc.commitment import Commitment
import time


client = Client("https://api.devnet.solana.com")

client.is_connected()

True

In [12]:
program_id = PublicKey("DdyiDbh71JnpYbxdA9i4ECcd573sTm8Uur2GCgcB3P5k")
# program_id_account_meta = AccountMeta(program_id, False, False)
sys_program_meta = AccountMeta(SYS_PROGRAM_ID, False, False)
spl_program_meta  = AccountMeta(TOKEN_PROGRAM_ID, False, False)
# program_id_account_meta

In [13]:
# client.request_airdrop(PublicKey("DxkiJXK31VEd4R6F44mQRszoUQugpPYMwL9ybcrnt22r"), 2000000000)

In [14]:
def getAccountInfo(client: Client, account_pk: PublicKey):
    """Fetch account information from RPC, parse out the data and deserialize"""
    comm = Commitment("confirmed")
    res = client.get_account_info(account_pk, comm, encoding='base64')
    data = res['result']
    if isinstance(data, dict):
        return accountSchema.parse(base64.urlsafe_b64decode(data['value']['data'][0]))
    else:
        raise AttributeError(f'Unknown RPC result {data}')

In [15]:
seed = bytes([125,215,212,54,213,98,45,123]*4)
payer_keypair = Keypair.from_seed(seed)
# client.request_airdrop(payer_keypair.public_key, 2*10**9)

In [16]:
mint_account = Keypair.generate()

seed1 = bytes([225,215,211,54,213,18,25,122]*4)
vault_keypair = Keypair.from_seed(seed1)

payer_account_meta = AccountMeta(payer_keypair.public_key, True, True)
vault_account_meta = AccountMeta(PublicKey("G473EkeR5gowVn8CRwTSDop3zPwaNixwp62qi7nyVf4z"), False, True)

rent_account_meta = AccountMeta(solana.sysvar.SYSVAR_RENT_PUBKEY, False, False)
spl_program_meta  = AccountMeta(TOKEN_PROGRAM_ID, False, False)


In [17]:
mint_account_meta = AccountMeta(mint_account.public_key, True, True)
associated_token_address = instructions.get_associated_token_address(payer_keypair.public_key, mint_account.public_key)
associated_account_meta = AccountMeta(associated_token_address, False, True)

(mint_authority, nonce) = PublicKey.find_program_address([b'cyrial_pda'], program_id)
(avatar_data_pda, avatar_pda_nonce) = PublicKey.find_program_address([b'avatar_data_pda', bytes(mint_account_meta.pubkey), bytes(associated_account_meta.pubkey)], program_id)
(sales_pda, sales_nonce) = PublicKey.find_program_address([b'sales_pda', bytes(program_id)], program_id)
metadata_program_pubkey = PublicKey("metaqbxxUerdq28cj1RbAWkYQm3ybzjb6a8bt518x1s")
(metadata_pda, metadata_nonce) = PublicKey.find_program_address([b'metadata', bytes(metadata_program_pubkey), bytes(mint_account_meta.pubkey)], metadata_program_pubkey)


avatar_data_pda_meta = AccountMeta(avatar_data_pda, False, True)
sysvar_clock_meta = AccountMeta(solana.sysvar.SYSVAR_CLOCK_PUBKEY, False, False)
sales_pda_meta = AccountMeta(sales_pda, False, True)
mint_authority_meta = AccountMeta(mint_authority, False, False)

accounts = [
    payer_account_meta, 
    vault_account_meta, 
    mint_account_meta, 
    rent_account_meta, 
    associated_account_meta,
    spl_program_meta,
    AccountMeta(SYS_PROGRAM_ID, False, False),
    mint_authority_meta,
    avatar_data_pda_meta,
    sales_pda_meta,
    AccountMeta(metadata_pda, False, True),
    sysvar_clock_meta,

    spl_program_meta,
    AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False),
    spl_program_meta,
    spl_program_meta,
    AccountMeta(metadata_program_pubkey, False, False),
    ]

mint_account_meta.pubkey

HzugB5CiTrSr3SZkN3p9cudYoA1R1swzgE68vpFwfQoP

In [18]:
    
transaction = Transaction()
transaction.add(TransactionInstruction(
    accounts,
    program_id,
    bytes([0])
))
f"https://explorer.solana.com/tx/{client.send_transaction(transaction, payer_keypair, mint_account)['result']}?cluster=devnet"

'https://explorer.solana.com/tx/4ya13A72f84aqNxQ7de6A8yqWVCiFHmcscxH4ZHDVmJT5qGhtNHeKTc7WvLBHMNGeQP12zBPkPbS5GcA57NcfuqQ?cluster=devnet'

In [19]:

accountSchema = CStruct(
    "vault_total" / F32,
    "counter" / U32,
)

(sales_pda, sales_nonce) = PublicKey.find_program_address([b'sales_pda', bytes(program_id)], program_id)
print(f"PDA: {sales_pda, sales_nonce}")

getAccountInfo(client,sales_pda)

PDA: (C3aVsFBFcMqzJQy6pxoMzJvq9hoHjq2rXtGNfxM7CGqJ, 255)


Container(vault_total=19.965702056884766, counter=12)

In [20]:
accountSchema = CStruct(
    "date_created" / U32,
    "unlockable_date" / U32,
    "numeration" /U32,
    "rarity" / U8,
    "level" / U8,
    "xp" / U32,
)


(locktime_pda, locktime_nonce) = PublicKey.find_program_address([b'locktime_pda', bytes(mint_account_meta.pubkey), bytes(associated_account_meta.pubkey)], program_id)
print(f"PDA: {avatar_data_pda, avatar_pda_nonce}")

getAccountInfo(client,avatar_data_pda)

PDA: (6CmbS5CQPXL3VNYs44XECCcYA3N8nnU79S1FRnVZiYV6, 253)


Container(date_created=1645374454, unlockable_date=1645374542, numeration=11, rarity=1, level=0, xp=0)

In [36]:
time.sleep(120)

new_accounts = [
    mint_account_meta,
    associated_account_meta,
    spl_program_meta,
    mint_authority_meta,
    avatar_data_pda_meta,
    sysvar_clock_meta,

    spl_program_meta
]

transaction = Transaction()
transaction.add(TransactionInstruction(
    new_accounts,
    program_id,
    bytes([1])
))
f"https://explorer.solana.com/tx/{client.send_transaction(transaction, payer_keypair, mint_account)['result']}?cluster=devnet"